# Trabalho 1 - Computação Gráfica

- João Pedro Rodrigues Freitas, 11316552
- Gabriel Akio Urakawa, 11795912
- Vinícius Santos Monteiro, 11932463

### Bibliotecas necessárias

In [806]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

Definindo a janela de visualização

In [807]:
class Window():
    def __init__(self, nome, altura=1600, largura=1200):
        self.nome = nome
        self.altura = altura
        self.largura = largura
        self.window = None

    # Inicializando a janela
    def init(self):
        glfw.init()
        glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
        self.window = glfw.create_window(self.largura, self.altura, self.nome, None, None)
        glfw.make_context_current(self.window)

    def bindKeyCursor(self, key_event, mouse_event):
        glfw.set_key_callback(self.window,key_event)
        glfw.set_cursor_pos_callback(self.window, mouse_event)

    def showWindow(self, lastX, lastY):
        glfw.show_window(self.window)
        glfw.set_cursor_pos(self.window, lastX, lastY)

    def getWindow(self):
        return self.window

Configurando o OpenGL.

In [808]:
class Shader():
    def __init__(self, vertex_code, fragment_code):
        self.vertex_code = vertex_code
        self.fragment_code = fragment_code
        self.program = None
        self.vertex = None
        self.fragment = None
        self.textures = None
        self.qtd_textures = None
        self.buffer = None

    def init(self):
        # Requisitando slot na GPU para os programas Vertex e Fragment Shaders
        self.program = glCreateProgram()
        self.vertex = glCreateShader(GL_VERTEX_SHADER)
        self.fragment = glCreateShader(GL_FRAGMENT_SHADER)

        # Associando código-fonte aos slots solicitados
        glShaderSource(self.vertex, self.vertex_code)
        glShaderSource(self.fragment, self.fragment_code)

        # Compilando o Vertex Shader
        glCompileShader(self.vertex)
        if not glGetShaderiv(self.vertex, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(self.vertex).decode()
            print(error)
            raise RuntimeError("Erro de compilacao do Vertex Shader")
        
        # Compilando o Fragment Shader
        glCompileShader(self.fragment)
        if not glGetShaderiv(self.fragment, GL_COMPILE_STATUS):
            error = glGetShaderInfoLog(self.fragment).decode()
            print(error)
            raise RuntimeError("Erro de compilacao do Fragment Shader")
        
        # Associando os programas compilados ao programa principal
        glAttachShader(self.program, self.vertex)
        glAttachShader(self.program, self.fragment)

        # Linkagem do programa
        glLinkProgram(self.program)
        if not glGetProgramiv(self.program, GL_LINK_STATUS):
            print(glGetProgramInfoLog(self.program))
            raise RuntimeError('Linking error')
        
        # Tornar o programa como o principal
        glUseProgram(self.program)


    def configureTextures(self, qtd_textures):
        glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
        glEnable( GL_BLEND )
        glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
        glEnable(GL_LINE_SMOOTH)
        glEnable(GL_TEXTURE_2D)
        
        self.qtd_textures = qtd_textures
        self.textures = glGenTextures(self.qtd_textures)


    def enable3D(self):
        glEnable(GL_DEPTH_TEST)


    def sendObjectsToGPU(self, objects):
        self.buffer = glGenBuffers(3)

        vertices = np.zeros(len(objects.getVertices()), [("position", np.float32, 3)])
        vertices['position'] = objects.getVertices()

        # Envia os vértices para a GPU
        glBindBuffer(GL_ARRAY_BUFFER, self.buffer[0])
        glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
        stride = vertices.strides[0]
        offset = ctypes.c_void_p(0)
        loc_vertices = glGetAttribLocation(self.program, "position")
        glEnableVertexAttribArray(loc_vertices)
        glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

        textures = np.zeros(len(objects.getTextures()), [("position", np.float32, 2)]) # duas coordenadas
        textures['position'] = objects.getTextures()
        # Envia as texturas para a GPU
        glBindBuffer(GL_ARRAY_BUFFER, self.buffer[1])
        glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
        stride = textures.strides[0]
        offset = ctypes.c_void_p(0)
        loc_texture_coord = glGetAttribLocation(self.program, "texture_coord")
        glEnableVertexAttribArray(loc_texture_coord)
        glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)
        
        normals = np.zeros(len(objects.getNormals()), [("position", np.float32, 3)]) # três coordenadas
        normals['position'] = objects.getNormals()
        #Envia a iluminação para a GPU
        glBindBuffer(GL_ARRAY_BUFFER, self.buffer[2])
        glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
        stride = normals.strides[0]
        offset = ctypes.c_void_p(0)
        loc_normals_coord = glGetAttribLocation(self.program, "normals")
        glEnableVertexAttribArray(loc_normals_coord)
        glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

    
    def getProgram(self):
        return self.program


Interagindo com vertex e fragment shaders.

In [809]:
def vertexCode():
    return """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = normals;            
        }
        """

def fragmentCode():
    return """
        //escolher entre as fontes utilzadas
        uniform int useLight1; // 1 para usar a luz #1, 2 para usar a luz #2,
        
        // parametros da iluminacao ambiente e difusa
        uniform vec3 lightPos1; // define coordenadas de posicao da luz #1
        uniform vec3 lightPos2; // define coordenadas de posicao da luz #2
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        
        // parametro com a cor da(s) fonte(s) de iluminacao
        vec3 lightColor1 = vec3(1.0, 0.0, 0.0);
        vec3 lightColor2 = vec3(1.0, 1.0, 1.0);

        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * vec3(1.0,1.0,1.0);             
        
            ////////////////////////
            // Luz #1
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm1 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // direcao da luz
            float diff1 = max(dot(norm1, lightDir1), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse1 = kd * diff1 * lightColor1; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir1 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir1 = reflect(-lightDir1, norm1); // direcao da reflexao
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ks * spec1 * lightColor1;    
            
            
            ////////////////////////
            // Luz #2
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm2 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(lightPos2 - out_fragPos); // direcao da luz
            float diff2 = max(dot(norm2, lightDir2), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse2 = kd * diff2 * lightColor2; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir2 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir2 = reflect(-lightDir2, norm2); // direcao da reflexao
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ks * spec2 * lightColor2;    
            
            ////////////////////////
            // Combinando as duas fontes
            ////////////////////////
            
            vec3 diffuse, specular;

            if (useLight1 == 1) {
                diffuse = diffuse1;
                specular = specular1;
            } else {
                diffuse = diffuse2;
                specular = specular2;
            }

            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

Carregamento modelos a partir de arquivos obj.

In [810]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])
            
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals
    
    return model


Carregando texturas a partir de arquivos.

In [811]:
def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    print(img_textura,img.mode)
    img_width = img.size[0]
    img_height = img.size[1]
    #image_data = img.tobytes("raw", "RGB", 0, -1)
    image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)

Classe para gerenciar os modelos e texturas.

In [812]:
class Objects():
    def __init__(self):
        self.vertices_list = []
        self.textures_coord_list = []
        self.normals_list = []    

    def loadObject(self, modelPath, modelName):
        modelo = load_model_from_file(modelPath)

        # inserindo vertices do modelo no vetor de vertices
        print(f'Processando modelo {modelName}. Vertice inicial: {len(self.vertices_list)}')

        for face in modelo['faces']:
            for vertice_id in face[0]:
                self.vertices_list.append( modelo['vertices'][vertice_id-1] )
            for texture_id in face[1]:
                self.textures_coord_list.append( modelo['texture'][texture_id-1])
            for normal_id in face[2]:
                self.normals_list.append( modelo['normals'][normal_id-1] )

        print(f'Processando modelo {modelName}. Vertice final: {len(self.vertices_list)}')

    def loadTexture(self, textureID, texturePath):
        # inserindo coordenadas de textura do modelo no vetor de texturas

        # carregando textura equivalente e definindo um id (buffer): use um id por textura!
        load_texture_from_file(textureID, texturePath)

    def getVertices(self):
        return self.vertices_list
    
    def getTextures(self):
        return self.textures_coord_list
    
    def getNormals(self):
        return self.normals_list

Funcões para desenhar os modelos, aplicando a matriz model.

In [813]:
def desenhaGrama(program, camera, x, z, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    
    # rotacao
    angle = 0;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = x; t_y = -8.0; t_z = z;
    
        
    
    # escala
    s_x = 2.0; s_y = 2.0; s_z = 2.0;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu  
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 0, 24) ## renderizando
     


In [814]:
def desenhaTardisFloor(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 0;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 32.6; t_y = -4.0; t_z = 10.0;
    
    # escala
    s_x = 10.0; s_y = 10.0; s_z = 10.0;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu  
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 24, 318-24) ## renderizando 

In [815]:
def desenhaTardis(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
        
    # rotacao
    angle = 0;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 20.0; t_y = -5.8; t_z = 10.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 2)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 318, 102534 - 318) ## renderizando 

In [816]:
def desenhaCasa(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 0;
    r_x = 2.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 20.0; t_y = -5.0; t_z = 50.0;
    
    # escala
    s_x = 3.0; s_y = 3.0; s_z = 3.0;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 102534, 105546 - 102534) ## renderizando 
    

In [817]:
def desenhaPato(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 270;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 20.0; t_y = -4; t_z = 10.0;
    
    # escala
    s_x = 0.05; s_y = 0.05; s_z = 0.05;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu         
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 105546, 140554 - 105546) ## renderizando 

#     105546
# Processando modelo duck.obj. Vertice final: 140554
    

In [818]:
def desenhaTable(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 0;
    r_x = 2.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 24.0; t_y = 4.3; t_z = 46.2;
    
    # escala
    s_x = 1.5; s_y = 1.5; s_z = 1.5;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu          
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 5)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 140554, 162558 - 140554) ## renderizando 
    

In [819]:
def desenhaLamp(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 270;
    r_x = 2.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 10.0; t_y = -4; t_z = 10;
    
    # escala
    s_x = 0.08; s_y = 0.08; s_z = 0.08;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu          
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 162558, 391486 - 162558) ## renderizando 

In [820]:
def desenhaBed(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 270;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 29.0; t_y = 4.1; t_z = 45.2;
    
    # escala
    s_x = 0.05; s_y = 0.05; s_z = 0.05;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu         
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 391486, 477214 - 391486) ## renderizando 

In [821]:
def desenhaDog(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 270;
    r_x = 1.0; r_y = 0.0; r_z = 0;
    
    # translacao
    t_x = 20.0; t_y = 4.1; t_z = 50.2;
    
    # escala
    s_x = 0.1; s_y = 0.1; s_z = 0.1;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu         
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    
    
    # desenha o modelo
    glDrawArrays(GL_QUADS, 477214, 621150 - 477214) ## renderizando 

In [822]:
def desenhaCeu(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    # rotacao
    angle = 0;
    r_x = 1.0; r_y = 0.0; r_z = 0.0;
    
    # translacao
    t_x = 50.0; t_y = 0.0; t_z = 50.0;
    
    # escala
    s_x = 150.0; s_y = 150.0; s_z = 150.0;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu       
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 9)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 621150, 633054 - 621150) ## renderizando 

In [823]:
def desenhaRot(program, camera, ka, kd, ks, ns, interno):
    if interno:
        useLight1 = 1 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    else:
        useLight1 = 2 
        glUniform1i(glGetUniformLocation(program, "useLight1"), useLight1)
    (r, (tx, tz), s) = camera.obj()
    # rotacao
    angle = r;
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 15.0+tx; t_y = -4.0; t_z = 15.0+tz;
    
    # escala
    s_x = 2+s; s_y = 2+s; s_z = 2+s;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    (ka_inc, kd_inc, ks_inc) = camera.ilumination()
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka+ka_inc) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_kd, kd+kd_inc) ### envia kd pra gpu  
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks+ks_inc) ### envia ks pra gpu         
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 10)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 633054, 633378 - 633054) ## renderizando 

In [824]:
def desenha_luz(program, camera, ka, kd, ks, ns):
    # aplica a matriz model
    angle = 0.0
    
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 24.0; t_y = 10.2; t_z = 44.5;
    
    # escala
    s_x = 0.5; s_y = 1.0; s_z = 0.5;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ns pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu            
    
    loc_light_pos = glGetUniformLocation(program, "lightPos1") # recuperando localizacao da variavel lightPos na GPU
    glUniform3f(loc_light_pos, t_x, t_y, t_z) ### posicao da fonte de luz
        
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 11)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 633378, 36) ## renderizando


In [825]:
def desenha_luzRot(program, camera, ka, kd, ks, ns):   
    (r, (tx, tz), s) = camera.obj()
    # aplica a matriz model
    angle = r
    
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 15.0 + tx; t_y = 0.3; t_z = 15.0 + tz;
    
    # escala
    s_x = 0.2+s; s_y = 0.2+s; s_z = 0.2+s;
    
    mat_model = camera.model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    
    #### define parametros de ilumincao do modelo
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ns pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu            
    
    loc_light_pos = glGetUniformLocation(program, "lightPos2") # recuperando localizacao da variavel lightPos na GPU
    glUniform3f(loc_light_pos, t_x, t_y, t_z) ### posicao da fonte de luz
        
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 12)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 633414, 36) ## renderizando
    

In [826]:
class Camera():
    def __init__(self, altura, largura):
        self.altura = altura
        self.largura = largura

        self.cameraPos   = glm.vec3(50.0,  0.0,  50.0);
        self.cameraFront = glm.vec3(0.0,  0.0, -1.0);
        self.cameraUp    = glm.vec3(0.0,  1.0,  0.0);
        
        self.ka_inc = 0.2
        self.kd_inc = 0.5
        self.ks_inc = 0.7

        self.polygonal_mode = False

        self.inc_fov = 0
        self.inc_near = 0
        self.inc_far = 0
        self.inc_view_up = 0

        self.firstMouse = True
        self.yaw = -90.0 
        self.pitch = 0.0
        self.lastX = largura/2
        self.lastY = altura/2
        
        self.rotObj = 0
        self.translationObjX = 0
        self.translationObjZ = 0
        self.scaleObj = 0

    def getLastX(self):
        return self.lastX
    
    def getLastY(self):
        return self.lastY
    
    def getPolygonalMode(self):
        return self.polygonal_mode

    def model(self, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
        angle = math.radians(angle)
        
        matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade
        
        # aplicando translacao
        matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
        
        # aplicando rotacao    
        matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
            
        
        # aplicando escala
        matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
        
        return np.array(matrix_transform).T # pegando a transposta da matriz (glm trabalha com ela invertida)

    def ilumination(self):
        return (self.ka_inc, self.kd_inc, self.ks_inc)
    
    def obj(self):
        return (self.rotObj, (self.translationObjX, self.translationObjZ), self.scaleObj)
    
    def view(self):
        mat_view = glm.lookAt(self.cameraPos, self.cameraPos + self.cameraFront, self.cameraUp);
        return np.array(mat_view)

    def projection(self):
        mat_projection = glm.perspective(glm.radians(45.0), self.largura/self.altura, 0.1, 1000.0)
        return np.array(mat_projection)

    def key_event(self, window,key,scancode,action,mods):
        print(key)
        if key == 66:
            self.inc_view_up += 0.1
            #cameraUp    = glm.vec3(0.0+inc_view_up,  1.0+inc_view_up,  0.0+inc_view_up);
        if key == 78: self.inc_near += 0.1
        if key == 77: self.inc_far -= 5
            

        cameraSpeed = 0.5
        if key == 82: # tecla R
            self.rotObj += 10
        if key == 69: # tecla E
            self.rotObj -= 10
        if key == 74: # tecla J
            self.scaleObj -= 0.1
        if key == 76: # tecla L
            self.scaleObj += 0.1
        if key == 262: # Seta para direita
            self.translationObjX += 0.1
        if key == 263: # Seta para esquerda
            self.translationObjX -= 0.1    
        if key == 264: # Seta para cima
            self.translationObjZ += 0.1
        if key == 265: # Seta para baixo
            self.translationObjZ -= 0.1   
        if key == 84: # tecla t
            self.kd_inc += 0.05    
        if key == 89: # tecla y
            self.kd_inc -= 0.05    
        if key == 85: # tecla u
            self.ka_inc += 0.05    
        if key == 73: # tecla i
            self.ka_inc -= 0.05    
        if key == 71: # tecla g
            self.ks_inc += 0.05    
        if key == 72: # tecla h
            self.ks_inc -= 0.05    
             
            
            
        
        if key == 87 and (action==1 or action==2): # tecla W
            self.cameraPos += cameraSpeed * self.cameraFront
        
        if key == 83 and (action==1 or action==2): # tecla S
            self.cameraPos -= cameraSpeed * self.cameraFront
        
        if key == 65 and (action==1 or action==2): # tecla A
            self.cameraPos -= glm.normalize(glm.cross(self.cameraFront, self.cameraUp)) * cameraSpeed
            
        if key == 68 and (action==1 or action==2): # tecla D
            self.cameraPos += glm.normalize(glm.cross(self.cameraFront, self.cameraUp)) * cameraSpeed
            
        if key == 80 and action==1 and self.polygonal_mode==True: # Tecla P
            self.polygonal_mode=False
        else:
            if key == 80 and action==1 and self.polygonal_mode==False:
                self.polygonal_mode=True
                
                
        if self.cameraPos[0] > 100:
            self.cameraPos[0] = 100
        elif self.cameraPos[0] < 0:
            self.cameraPos[0] = 0
        if self.cameraPos[1] < 0:
            self.cameraPos[1] = 0
        if self.cameraPos[1] > 100:
            self.cameraPos[1] = 100
        if self.cameraPos[2] > 100:
            self.cameraPos[2] = 100
        elif self.cameraPos[2] < 0:
            self.cameraPos[2] = 0

    def mouse_event(self, window, xpos, ypos):
        if self.firstMouse:
            self.lastX = xpos
            self.lastY = ypos
            self.firstMouse = False

        xoffset = xpos - self.lastX
        yoffset = self.lastY - ypos
        self.lastX = xpos
        self.lastY = ypos

        sensitivity = 0.3 
        xoffset *= sensitivity
        yoffset *= sensitivity

        self.yaw += xoffset;
        self.pitch += yoffset;

        
        if self.pitch >= 90.0: self.pitch = 90.0
        if self.pitch <= -90.0: self.pitch = -90.0

        front = glm.vec3()
        front.x = math.cos(glm.radians(self.yaw)) * math.cos(glm.radians(self.pitch))
        front.y = math.sin(glm.radians(self.pitch))
        front.z = math.sin(glm.radians(self.yaw)) * math.cos(glm.radians(self.pitch))
        self.cameraFront = glm.normalize(front)

In [827]:
def main():
    altura_janela = 1600
    largura_janela = 1200

    janela = Window("Trabalho 1", altura_janela, largura_janela)
    janela.init()

    shader = Shader(vertexCode(), fragmentCode())
    shader.init()
    shader.configureTextures(qtd_textures=15) # 10 texturas
    
    objetos = Objects()

    # # Fazer leitura dos objetos e texturas
    
    
    objetos.loadObject('minecraft-grass-block/Grass_Block.obj', 'Grass_Block.obj')
    objetos.loadTexture(0, 'minecraft-grass-block/Grass_Block_TEX.png')
    
    objetos.loadObject('tardis_floor/Sci-Fi-Floor-1-OBJ.obj', 'Sci-Fi-Floor-1-OBJ.obj')
    objetos.loadTexture(1, 'tardis_floor/Sci-Fi_Floor_textures/Sci-Fi Floor textures/Sci-Fi-Floor-Diffuse.tga')

    objetos.loadObject('police_box_obj/police box.obj', 'police box.obj')
    objetos.loadTexture(2, 'police_box_obj/police box.png')
    
    objetos.loadObject('house/medieval.obj', 'medieval.obj')
    objetos.loadTexture(3, 'house/diff.png')

    objetos.loadObject('duck/duck.obj', 'duck.obj')
    objetos.loadTexture(4, 'duck/duck.jpg')
    
    objetos.loadObject('Table/Table.obj', 'Table.obj')
    objetos.loadTexture(5, 'Table/Drawer Texture(Final).png')
    
    objetos.loadObject('lamp/lamp.obj', 'lamp.obj')
    objetos.loadTexture(6, 'lamp/Lamp.jpg')
    
    objetos.loadObject('Bed/BED.obj', 'BED.obj')
    objetos.loadTexture(7, 'Bed/BED.jpg')
    
    objetos.loadObject('Dog/Dog.obj', 'Dog.obj')
    objetos.loadTexture(8, 'Dog/Dog.jpg')
    
    objetos.loadObject('Ceu/Ceu.obj', 'Ceu.obj')
    objetos.loadTexture(9, 'Ceu/Ceu.jpg')
    
    objetos.loadObject('Rot/Rot.obj', 'Rot.obj')
    objetos.loadTexture(10, 'Rot/Rot.png')
    
    objetos.loadObject('Luz/Luz.obj', 'Luz.obj')
    objetos.loadTexture(11, 'Luz/Luz.png')
    
    objetos.loadObject('Luz/Luz.obj', 'Luz.obj')
    objetos.loadTexture(12, 'Luz/Luz.png')

    shader.sendObjectsToGPU(objetos)

    camera = Camera(altura=altura_janela, largura=largura_janela)

    janela.bindKeyCursor(camera.key_event, camera.mouse_event)
    janela.showWindow(camera.getLastX(), camera.getLastY())

    shader.enable3D()

    rotacao_inc = 0
    while not glfw.window_should_close(janela.getWindow()):
        glfw.poll_events() 
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        glClearColor(1.0, 1.0, 1.0, 1.0)

        if camera.getPolygonalMode() == True:
            glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
        if camera.getPolygonalMode() == False:
            glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
        
        rotacao_inc += 0.1

        geral_ka = 0.5
        geral_kd = 0.6
        geral_ks = 0.7
        geral_ns = 100
        
        for i in range(0, 101, 4):
            for j in range(0, 101, 4):
                desenhaGrama(shader.getProgram(), camera, i, j, geral_ka, geral_kd-0.3, geral_ks-0.3, geral_ns-50, False)

        desenhaTardisFloor(shader.getProgram(), camera, geral_ka, geral_kd+0.1, geral_ks+0.1, geral_ns+50, False)

        desenhaTardis(shader.getProgram(), camera, geral_ka, geral_kd+0.05, geral_ks+0.15, geral_ns+200, False)
        
        desenhaCasa(shader.getProgram(), camera, geral_ka, geral_kd-0.1, geral_ks-0.2, geral_ns-40, True)

        desenhaPato(shader.getProgram(), camera, geral_ka, geral_kd-0.18, geral_ks-0.23, geral_ns-45, False)
        
        desenhaTable(shader.getProgram(), camera, geral_ka, geral_kd-0.21, geral_ks-0.31, geral_ns-38, True)
        
        desenhaLamp(shader.getProgram(), camera, geral_ka, geral_kd+0.2, geral_ks+0.8, geral_ns+130, True)
        
        desenhaBed(shader.getProgram(), camera, geral_ka, geral_kd-0.11, geral_ks-0.12, geral_ns-40, True)
        
        desenhaDog(shader.getProgram(), camera, geral_ka, geral_kd+0.1, geral_ks-0.3, geral_ns-20, True)
        
        desenhaCeu(shader.getProgram(), camera, geral_ka, geral_kd, geral_ks, geral_ns, False)
   
        desenhaRot(shader.getProgram(), camera, geral_ka, geral_kd+0.05, geral_ks-0.15, geral_ns-30, False)
        
        desenha_luz(shader.getProgram(), camera, 1, 1, 1, 1000)
        
        desenha_luzRot(shader.getProgram(), camera, 1, 1, 1, 1000)
        
        mat_view = camera.view()
        loc_view = glGetUniformLocation(shader.getProgram(), "view")
        glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

        mat_projection = camera.projection()
        loc_projection = glGetUniformLocation(shader.getProgram(), "projection")
        glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
        
        loc_view_pos = glGetUniformLocation(shader.getProgram(), "viewPos") # recuperando localizacao da variavel viewPos na GPU
        glUniform3f(loc_view_pos, camera.cameraPos[0], camera.cameraPos[1], camera.cameraPos[2]) ### posicao da camera/observador (x,y,z)
        
        glfw.swap_buffers(janela.getWindow())

    glfw.terminate()


Definindo a chamada do método main para inicializar o programa.

In [828]:
if __name__ == "__main__":
    main()

Processando modelo Grass_Block.obj. Vertice inicial: 0
Processando modelo Grass_Block.obj. Vertice final: 24
minecraft-grass-block/Grass_Block_TEX.png RGBA
Processando modelo Sci-Fi-Floor-1-OBJ.obj. Vertice inicial: 24
Processando modelo Sci-Fi-Floor-1-OBJ.obj. Vertice final: 318
tardis_floor/Sci-Fi_Floor_textures/Sci-Fi Floor textures/Sci-Fi-Floor-Diffuse.tga RGB
Processando modelo police box.obj. Vertice inicial: 318
Processando modelo police box.obj. Vertice final: 102534
police_box_obj/police box.png RGBA
Processando modelo medieval.obj. Vertice inicial: 102534
Processando modelo medieval.obj. Vertice final: 105546
house/diff.png RGBA
Processando modelo duck.obj. Vertice inicial: 105546
Processando modelo duck.obj. Vertice final: 140554
duck/duck.jpg RGB
Processando modelo Table.obj. Vertice inicial: 140554
Processando modelo Table.obj. Vertice final: 162558
Table/Drawer Texture(Final).png RGBA
Processando modelo lamp.obj. Vertice inicial: 162558
Processando modelo lamp.obj. Vertic